### Submission
- MultiOutput Bayesian Optimization을 LGBM과 Catboost모델에 각각 적용시켰을 때, 가장 점수가 높게 나온 모델을 각각의 알고리즘에서 2개씩 뽑아 총 4개를 앙상블 시킨 결과가 제일 좋았음.
- 앙상블을 시킬 때, 각각의 알고리즘에서 성능이 가장 좋은 모델에 가중치를 2/3, 두번째로 성능이 좋은 모델에 가중치를 1/3 적용시켰음.

### 패키지 로딩

In [1]:
# Data Handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# fix seed
import random
import os
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42)

# Modeling
from sklearn.multioutput import MultiOutputRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

c:\Users\user\miniconda3\envs\env\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### 데이터 로딩 및 처리

In [3]:
### Load Data
train = pd.read_csv("C:/LG_Aimers_NEW/data/train.csv")
test = pd.read_csv("C:/LG_Aimers_NEW/data/test.csv")
submission = pd.read_csv("C:/LG_Aimers_NEW/data/sample_submission.csv")

### Preprocessing
test = test.drop(columns = ["ID"])
X = train.iloc[:,1:57]
y = train.iloc[:,57:]

### 모델링

In [6]:
cat1 = MultiOutputRegressor(CatBoostRegressor(depth = int(9.981652216959748),bagging_temperature = 6.410670492107986, learning_rate =  0.022984684781070068, subsample = 0.5938399056973557, random_state=1339,verbose = 0)).fit(X, y)
cat2 = MultiOutputRegressor(CatBoostRegressor(depth = int(8.842),bagging_temperature = 7.806, learning_rate =  0.02811, subsample = 0.8751, random_state=1339,verbose = 0)).fit(X, y)
lgbm1 = MultiOutputRegressor(LGBMRegressor(num_leaves = int(241.3), learning_rate = 0.01618,n_estimators = int(404.4), subsample =0.2837, colsample_bytree =  0.3572)).fit(X, y)
lgbm2 = MultiOutputRegressor(LGBMRegressor(num_leaves = int(409.5), learning_rate = 0.0216,n_estimators = int(319.7), subsample =0.04784, colsample_bytree =   0.4813)).fit(X, y)

In [7]:
cat1_predict = cat1.predict(test)
cat2_predict = cat2.predict(test)
lgbm1_predict = lgbm1.predict(test)
lgbm2_predict = lgbm2.predict(test)

### 예측

In [8]:
cat1_predict = pd.DataFrame(cat1_predict)
cat2_predict = pd.DataFrame(cat2_predict)

In [9]:
lgbm1_predict = pd.DataFrame(lgbm1_predict)
lgbm2_predict = pd.DataFrame(lgbm2_predict)

In [10]:
cat1_predict.columns = y.columns
lgbm1_predict.columns = y.columns
cat2_predict.columns = y.columns
lgbm2_predict.columns = y.columns

### 제출

In [20]:
result = pd.DataFrame()

In [21]:
for i in y.columns :
    result[i] = ((cat1_predict[i] + lgbm1_predict[i])/2)*(2/3) + ((cat2_predict[i] + lgbm2_predict[i])/2)*(1/3)

In [22]:
ID = submission["ID"]

In [23]:
result = pd.concat([ID,result],axis = 1)

In [24]:
result.to_csv("C:/LG_Aimers_NEW/submission/submit_cat12_lgbm12_esb_csv.csv",index = False)